In [55]:
from datasets import load_dataset
import google.generativeai as genai

from transformers import AutoTokenizer

import json
import re

In [ ]:
ds = load_dataset("elricwan/HarryPotter")
text = ds['train'][0]['content']

In [ ]:
GOOGLE_API_KEY = ""
genai.configure(api_key=GOOGLE_API_KEY)

for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinki

In [ ]:
def get_prompt(history, context):
    template = """
    You are reading a book. The portion you have already read is referred to as the **history**, and a new **context** (an excerpt from a later part of the book) is provided. 

    Your task is to generate **10 multiple-choice questions** based on the *context*. However, the **correct answers to these questions must depend on the information provided in the history**—not just the context alone.

    Each question should have:
    - A clear and concise question stem
    - Four answer choices (A, B, C, D)
    - One correct answer that relies on the history
    - A brief explanation of why the correct answer is correct (optional but recommended for clarity)

    Here are the inputs:

    **History**:  
    {history}

    **Context**:  
    {context}

    Now, generate 50 multiple-choice questions as described above and return your response strictly in JSON format.
    """
    return template.format(history=history, context=context)

In [20]:
chapters = text.split("CHAPTER")

In [31]:
history = chapters[1]
context = chapters[2]

In [24]:
test_model_name = "Qwen/Qwen2.5-1.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(test_model_name)

In [33]:
history_tokens = tokenizer.encode(history)
context_tokens = tokenizer.encode(context)
print(f"# of history tokens : {len(history_tokens)}")
print(f"# of context tokens : {len(context_tokens)}")

# of history tokens : 6191
# of context tokens : 4453


In [40]:
# model = genai.GenerativeModel('gemini-1.5-flash-8b')
model = genai.GenerativeModel('gemini-1.5-pro')

In [47]:
prompt = get_prompt(history, context)
response = model.generate_content(prompt)
print(response.text)

```json
[
  {
    "question": "Why does Harry live with the Dursleys?",
    "options": {
      "A": "He is an orphan and they are his closest living relatives.",
      "B": "He chose to live with them after his parents' death.",
      "C": "They adopted him after his parents abandoned him.",
      "D": "He is their biological son, but they keep it a secret."
    },
    "answer": "A",
    "explanation": "The history explains that Harry's parents, the Potters, died and the Dursleys are his only remaining family."
  },
  {
    "question": "Why are the Dursleys reluctant to take Harry to the zoo?",
    "options": {
      "A": "They dislike zoos and prefer other outings.",
      "B": "They are worried Harry will misbehave in public.",
      "C": "They cannot afford the zoo entrance fees.",
      "D": "They fear Harry's presence will somehow ruin Dudley's birthday."
    },
    "answer": "D",
    "explanation": "The context shows they're mainly concerned with keeping Dudley happy and consider

In [80]:
output = response.text 

In [ ]:
pattern = r'\[(.*?)\]'
result = re.search(pattern, output, re.DOTALL) 
if result:
    json_content = "[" + result.group(1) + "]"
    print(json_content)

In [82]:
try:
    # Parse the JSON to ensure it's valid
    json_data = json.loads(json_content)
    
    # Write to file with nice formatting
    with open('questions.json', 'w', encoding='utf-8') as f:
        json.dump(json_data, f, indent=2)
    
    print("JSON successfully saved to questions.json")
except json.JSONDecodeError as e:
    print(f"Error: Invalid JSON: {e}")

JSON successfully saved to questions.json
